In [1]:
from pptx import Presentation
prs = Presentation("b.pptx")

In [2]:
s = [a for a in prs.slides]
len(s)

70

In [3]:
for slide in s[18:44]:
    for shape in slide.shapes:
        print(shape.element.getchildren)

<bound method _Element.getchildren of <Element {http://schemas.openxmlformats.org/presentationml/2006/main}sp at 0x1517ee820e0>>
<bound method _Element.getchildren of <Element {http://schemas.openxmlformats.org/presentationml/2006/main}sp at 0x1517de5de50>>
<bound method _Element.getchildren of <Element {http://schemas.openxmlformats.org/presentationml/2006/main}sp at 0x1517d8a09f0>>
<bound method _Element.getchildren of <Element {http://schemas.openxmlformats.org/presentationml/2006/main}sp at 0x1517de451d0>>
<bound method _Element.getchildren of <Element {http://schemas.openxmlformats.org/presentationml/2006/main}sp at 0x1517de5de50>>
<bound method _Element.getchildren of <Element {http://schemas.openxmlformats.org/presentationml/2006/main}graphicFrame at 0x1517d8a0ae0>>
<bound method _Element.getchildren of <Element {http://schemas.openxmlformats.org/presentationml/2006/main}graphicFrame at 0x1517de451d0>>
<bound method _Element.getchildren of <Element {http://schemas.openxmlformats

In [16]:
shape.element.grpSpPr.get_or_add_effectLst

<bound method ZeroOrOne._add_get_or_adder.<locals>.get_or_add_child of <Element {http://schemas.openxmlformats.org/presentationml/2006/main}grpSpPr at 0x25ee13086d0>>

<bound method _Element.getchildren of <Element {http://schemas.openxmlformats.org/presentationml/2006/main}grpSp at 0x25ee123f400>>

In [4]:
gaps = {}
recs = {}
titles = []
for slide in s[18:44]:
    i=0
    gap = []
    rec = []
    for shape in slide.shapes:
        if shape.has_text_frame:
            p = shape.text_frame.paragraphs
            for paragraph in shape.text_frame.paragraphs:
                for run in paragraph.runs:
                    a = run.text
                    if a=="" or a==" " or "Draft" in a or "review" in a:
                        pass
                    elif " " in a:
                        title = a   
                        titles.append(a)
            continue
        elif not shape.has_table:
            continue
        i+=1
        if i==1:
            continue   
        tbl = shape.table

        row_count = len(tbl.rows)
        col_count = len(tbl.columns)
        for r in range(1, row_count):
            if i==2:
                for c in range(1, col_count):
                    cell = tbl.cell(r,c)
                    paragraphs = cell.text_frame.paragraphs 
                    for paragraph in paragraphs:
                        for run in paragraph.runs:
                            if run.text == ' ' or run.text in [' - ', ' – ', '- ', '– ', ' -', ' –']:
                                pass
                            else:
                                gap.append(run.text)
            else:
                for c in range(0, col_count):
                    cell = tbl.cell(r,c)
                    paragraphs = cell.text_frame.paragraphs 
                    for paragraph in paragraphs:
                        for run in paragraph.runs:
                            rec.append(run.text)
    if gap:
        gaps[title] = gap
        recs[title] = rec
print(gaps)
print(recs)

{'Strategy & Metrics': ['SM1', '–', ' Risk tolerance and threshold levels have not been determined at an enterprise level. This might lead to inconsistent risk categorization and treatment of identified vulnerabilities. ', 'SM2', ' – Application security strategic plan does not incorporate risk tolerance levels ', 'and are not communicated to the stakeholders periodically.', 'SM3', ' – Application security program lacks metrics and KPIs/KRIs to measure its ', 'effectiveness.'], 'Policy & Compliance': ['PC1', '– TS currently relies on manual reviews to ensure compliance and is operating with reduced efficiency due to limited support staff.', 'PC2 ', '– A lack of security experts has reduced the capability for the organization to enforce policies, compliance, and adoption within TS.', 'PC3', ' – Application security run-books lacks technology-specific implementation guidance.'], 'Education & Guidance': ['EG1', ' – Training feedback is not always requested which limits improvement opportu

In [5]:
g = []
for k, v in gaps.items():
    for _ in range(int(len(v)/2)):
        g.append(['', k, v[0], v[1]])
        del v[:2]
g


[['', 'Strategy & Metrics', 'SM1', '–'],
 ['',
  'Strategy & Metrics',
  ' Risk tolerance and threshold levels have not been determined at an enterprise level. This might lead to inconsistent risk categorization and treatment of identified vulnerabilities. ',
  'SM2'],
 ['',
  'Strategy & Metrics',
  ' – Application security strategic plan does not incorporate risk tolerance levels ',
  'and are not communicated to the stakeholders periodically.'],
 ['',
  'Strategy & Metrics',
  'SM3',
  ' – Application security program lacks metrics and KPIs/KRIs to measure its '],
 ['',
  'Policy & Compliance',
  'PC1',
  '– TS currently relies on manual reviews to ensure compliance and is operating with reduced efficiency due to limited support staff.'],
 ['',
  'Policy & Compliance',
  'PC2 ',
  '– A lack of security experts has reduced the capability for the organization to enforce policies, compliance, and adoption within TS.'],
 ['',
  'Policy & Compliance',
  'PC3',
  ' – Application security 

In [6]:
r = []
i=1
for k, v in recs.items():
    for _ in range(int(len(v)/2)):
        r.append(['', k, f"RI{i}", v[0], v[1]])
        del v[:2]
        i+=1
r

[['',
  'Strategy & Metrics',
  'RI1',
  'R&S should define, formalize and track the metrics & KPIs/ KRIs for the application security program to measure its effectiveness. TS should adopt and report on the applicable metrics & KPI/KRIs to R&S.',
  'SM3'],
 ['',
  'Strategy & Metrics',
  'RI2',
  'R&S should define risk tolerance and threshold levels for the organization and effectively communicate them to TS.  The application security strategic plan should be updated to reflect the defined risk tolerance and threshold levels along with the feedback from the KPIs/KRIs.',
  'SM1, SM2'],
 ['',
  'Policy & Compliance',
  'RI3',
  'TS should define and track application compliance metrics to identify gaps and prioritize remediation efforts. Metrics should be reported in the monthly stakeholder meeting. TS should use information from compliance reports to ensure adherence and close gaps. ',
  'PC1'],
 ['',
  'Policy & Compliance',
  'RI4',
  'R&S should add a BISO to partner with TS to be t

In [198]:
import xlsxwriter

with xlsxwriter.Workbook('test.xlsx') as workbook:
    worksheet1 = workbook.add_worksheet()

    for row_num, data in enumerate(g):
        worksheet1.write_row(row_num, 0, data)
        
    worksheet2 = workbook.add_worksheet()

    for row_num, data in enumerate(r):
        worksheet2.write_row(row_num, 0, data)